In [7]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    AdamW,
)


In [8]:

# Load data
df = pd.read_csv("/home/liorkob/M.Sc/thesis/data/drugs_3k/tagged_punishment_range_sentences.csv")
texts = df["text"].astype(str).tolist()

# Load HeBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("avichr/heBERT")

# Tokenize and get lengths
token_lengths = [len(tokenizer.encode(t, truncation=False)) for t in texts]

# Stats
max_len = max(token_lengths)
min_len = min(token_lengths)
avg_len = sum(token_lengths) / len(token_lengths)

print(f"📏 Max tokens: {max_len}")
print(f"📉 Min tokens: {min_len}")
print(f"📊 Avg tokens: {avg_len:.2f}")


📏 Max tokens: 1287
📉 Min tokens: 7
📊 Avg tokens: 88.76


In [12]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    AdamW,
)

# Load data
df = pd.read_csv("/home/liorkob/M.Sc/thesis/data/drugs_3k/tagged_punishment_range_sentences.csv")
texts = df["text"].astype(str).tolist()
labels = df["Tag"].astype(int).tolist()


train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts,
    labels,
    test_size=0.2,
    stratify=labels,
    random_state=42
)


# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("avichr/heBERT")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Dataset
class PunishmentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = PunishmentDataset(train_encodings, train_labels)
val_dataset = PunishmentDataset(val_encodings, val_labels)

# Grid setup
optimizers = {
    "adamw": lambda model, lr: AdamW(model.parameters(), lr=lr),
    "adam": lambda model, lr: torch.optim.Adam(model.parameters(), lr=lr),
    "sgd": lambda model, lr: torch.optim.SGD(model.parameters(), lr=lr),
    "adagrad": lambda model, lr: torch.optim.Adagrad(model.parameters(), lr=lr),
}
epochs_list = [2, 3, 4]
learning_rates = [5e-5, 3e-5, 1e-5]

# Best tracker
best_f1 = 0
best_model_info = ""

def compute_metrics(pred):
    preds = np.argmax(pred.predictions, axis=1)
    labels = pred.label_ids
    f1 = f1_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    return {
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

# Grid search
for opt_name, opt_fn in optimizers.items():
    for lr in learning_rates:
        for epochs in epochs_list:
            print(f"\n🔍 Trying: {opt_name}, LR={lr}, Epochs={epochs}")
            
            model = AutoModelForSequenceClassification.from_pretrained("avichr/heBERT", num_labels=2)
            optimizer = opt_fn(model, lr)
            
            args = TrainingArguments(
                output_dir=f"./results_{opt_name}_{lr}_{epochs}",
                num_train_epochs=epochs,
                per_device_train_batch_size=8,
                per_device_eval_batch_size=8,
                evaluation_strategy="epoch",
                save_strategy="no",
                logging_strategy="no",
                report_to="none"
            )

            trainer = Trainer(
                model=model,
                args=args,
                train_dataset=train_dataset,
                eval_dataset=val_dataset,
                compute_metrics=compute_metrics,
                optimizers=(optimizer, None)
            )

            trainer.train()
            metrics = trainer.evaluate()
            f1, precision, recall = metrics["eval_f1"], metrics["eval_precision"], metrics["eval_recall"]

            print(f"✅ F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")

            if f1 > best_f1:
                best_f1 = f1
                best_model_info = f"{opt_name} | LR={lr} | Epochs={epochs} | F1={f1:.4f} | Precision={precision:.4f} | Recall={recall:.4f}"
                model.save_pretrained("best_model")
                tokenizer.save_pretrained("best_model")

print(f"\n🏆 Best model:\n{best_model_info}")




🔍 Trying: adamw, LR=5e-05, Epochs=2


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at avichr/heBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/liorkob/.conda/envs/new_env/lib/python3.9/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/liorkob/.conda/envs/new_env/lib/python3.9/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelis

Epoch,Training Loss,Validation Loss,F1,Precision,Recall
1,No log,0.207119,0.905109,0.939394,0.873239
2,No log,0.190336,0.930556,0.917808,0.943662


✅ F1: 0.9306 | Precision: 0.9178 | Recall: 0.9437

🔍 Trying: adamw, LR=5e-05, Epochs=3


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at avichr/heBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/liorkob/.conda/envs/new_env/lib/python3.9/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/liorkob/.conda/envs/new_env/lib/python3.9/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelis

...

Epoch,Training Loss,Validation Loss,F1,Precision,Recall
1,No log,0.330328,0.859375,0.964912,0.774648
2,No log,0.258846,0.877698,0.897059,0.859155
3,No log,0.243575,0.887324,0.887324,0.887324


✅ F1: 0.8873 | Precision: 0.8873 | Recall: 0.8873

🔍 Trying: adagrad, LR=1e-05, Epochs=4


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at avichr/heBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/liorkob/.conda/envs/new_env/lib/python3.9/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss,F1,Precision,Recall
1,No log,0.298555,0.870229,0.950000,0.802817
2,No log,0.222720,0.875000,0.863014,0.887324
3,No log,0.199802,0.868966,0.851351,0.887324
4,No log,0.194455,0.868966,0.851351,0.887324


✅ F1: 0.8690 | Precision: 0.8514 | Recall: 0.8873

🏆 Best model:
adamw | LR=5e-05 | Epochs=2 | F1=0.9306 | Precision=0.9178 | Recall=0.9437


In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report

# 1. Load model and tokenizer
model_path = "best_model"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()  # Set to evaluation mode

# 2. Tokenize validation texts again
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")

# 3. Predict in batches
with torch.no_grad():
    outputs = model(**val_encodings)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).numpy()

# 4. Print predictions vs. ground truth
for i, (text, real, pred) in enumerate(zip(val_texts, val_labels, predictions)):
    print(f"\n📝 Text {i+1}:\n{text}\nReal: {real} | Predicted: {pred}")

# 5. Optional: Classification report
print("\n📊 Classification Report:")
print(classification_report(val_labels, predictions, digits=4))



📝 Text 1:
מאסר על תנאי לתקופה של 4 חודשים, והתנאי הוא שלא יעבור כל עבירה מסוג עוון לפי פקודת הסמים המסוכנים, למשך 3 שנים מהיום;
Real: 0 | Predicted: 0

📝 Text 2:
עיון בפסיקה מלמד אפוא, כי מתחם העונש ההולם את מעשיו של הנאשם בנסיבותיהם, מחייב עונש מאסר בפועל, כשהמתחם נע בין 12 ל-24 חודשי מאסר בפועל. בכל המקרים בהם הושת עונש מאסר בעבודות שירות, הדבר נעשה בסטייה ממתחם העונש ההולם, בשל הליך שיקום משמעותי ובצירוף נסיבות חריגות נוספות. כאמור, בת"פ (מחוזי-ת"א) 38450-09-12 בעניין סקוט, על אף השיקום המשמעותי שהנאשם עבר, ומדובר היה בנאשם בן 61 שצרך סמים מגיל 45 בשל מצב נפשי כתוצאה מנטייתו המינית, ואף ששם הוסכם כי הסם נרכש לשימוש עצמי, בית המשפט הקפיד לציין כי לא היה בכל אלו כדי להחריג את הנאשם ממתחם העונש ההולם המחייב עונש מאסר, והוא עשה זאת רק כי לכל האמור הצטרפו שיקולים רפואיים והומניטאריים מיוחדים. וכאמור, מדובר היה במי שעבר כברת דרך משמעותית בהליך השיקום שהחל מיוזמתו, וגילה מחויבות ומוטיבציה גבוהה להצליח בו. איני סבור כי בענייננו ניתן לומר כי הנאשם עבר הליך שיקומי. בוודאי לא משמעותי. אדרבה, 